In [192]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [193]:
dem = pd.read_csv('/home/stephen/Classes/Big Data Analytics/Final Project/seperated_sets/gen_dem.csv')
rep = pd.read_csv('/home/stephen/Classes/Big Data Analytics/Final Project/seperated_sets/gen_rep.csv')
dem = dem[dem['Party'] == 'Democrat']

gen = pd.concat([dem, rep], ignore_index=True)

In [194]:
def tweet_to_words(raw_tweet):
    letters_only = re.sub("[^a-zA-Z]", " ",raw_tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    return( " ".join( meaningful_words )) 

def clean_tweet_length(raw_tweet):
    letters_only = re.sub("[^a-zA-Z]", " ",raw_tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    return(len(meaningful_words)) 

In [195]:
gen['Party'] = np.where((gen['Party'] == 'Democrat'), 1, 0)
gen['Party'].value_counts()

0    44392
1    42068
Name: Party, dtype: int64

In [196]:
gen['clean_tweet']=gen['Tweet'].apply(lambda x: tweet_to_words(x))
gen['Tweet_length']=gen['Tweet'].apply(lambda x: clean_tweet_length(x))
gen['clean_tweet'] = gen['clean_tweet'].str.replace('RT','')

In [197]:
from sklearn.model_selection import train_test_split
from nltk.tokenize.casual import TweetTokenizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

tokenizer = TweetTokenizer(reduce_len=True)
X_train = gen['clean_tweet']
y_train = gen['Party']
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenizer.tokenize)
X_train = [str (item) for item in X_train]
X_train = tfidf_vectorizer.fit_transform(X_train)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=9)

In [198]:
print(X_train.shape)
nb = MultinomialNB().fit(X_train, y_train)
preds = nb.predict(X_test)
print('NB Accuracy: {}'.format(str(round(accuracy_score(y_test, preds), 4))))

(69168, 129919)
NB Accuracy: 0.795


In [199]:
import pickle

with open('NB_Model.pkl', 'wb') as file:
    pickle.dump(nb, file)

In [200]:
import pickle

with open('tfidf_Model.pkl', 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)